In [1]:
import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
# complex valued tensor class
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, print_pde
from RegscorePy.bic import bic

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N = 5000

DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = to_column_vector(Exact_u)
v_star = to_column_vector(Exact_v)

idx_x = np.random.choice(X_star.shape[0], N, replace=False)

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx_x, :], True).to(device)
u_train = to_tensor(u_star[idx_x, :], False).to(device)
v_train = to_tensor(v_star[idx_x, :], False).to(device)

feature_names = ['hf', '|hf|', 'h_x', 'h_xx', 'h_xxx']

You're running on cpu


In [3]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )

complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

complex_model.load_state_dict(cpu_load("./saved_path_inverse_nls/NLS_cpinn_model.pth"))

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


<All keys matched successfully>

In [4]:
class ComplexPhysicsInformedNN(nn.Module):
    def __init__(self, model, lb, ub, scale=False):
        super(ComplexPhysicsInformedNN, self).__init__()
        self.model = model
        self.lb = lb
        self.ub = ub
        self.scale = scale
    
    def forward(self, X):
        if self.scale: 
            return self.model(self.neural_net_scale(X))
        return self.model(X)

    def predict(self, X_test):
        return CplxToReal()(self.forward(self.preprocess(*dimension_slicing(X_test))))
    
    def neural_net_scale(self, inp):
        return (2.0*(inp-self.lb)/(self.ub-self.lb))-1.0

    def preprocess(self, spatial, time):
        return cat(spatial, time)
    
    def loss(self, X_f, X0, h0, X_lb, X_ub):
        loss = self.net_f(*dimension_slicing(X_f))
        h0_pred = self.predict(X0); u0 = h0_pred[:, 0:1]; v0 = h0_pred[:, 1:2]
        loss += F.mse_loss(u0, h0[:, 0:1])+F.mse_loss(v0, h0[:, 1:2])
        u_lb, v_lb, u_lb_x, v_lb_x = self.net_h(*dimension_slicing(X_lb))
        u_ub, v_ub, u_ub_x, v_ub_x = self.net_h(*dimension_slicing(X_ub))
        loss += F.mse_loss(u_lb, u_ub)
        loss += F.mse_loss(v_lb, v_ub)
        loss += F.mse_loss(u_lb_x, u_ub_x)
        loss += F.mse_loss(v_lb_x, v_ub_x)
        return loss
    
    def net_h(self, x, t):
        X = cat(x, t)
        h = self.forward(X)
        u = h.real
        v = h.imag
        return u, v, self.diff(u, x), self.diff(v, x)
    
    def net_f(self, x, t):
        u, v, u_x, v_x = self.net_h(x, t)
        u_t, v_t = self.diff(u, t), self.diff(v, t)
        u_xx, v_xx = self.diff(u_x, x), self.diff(v_x, x)
        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u
        return (f_u**2).mean()+(f_v**2).mean()

    def diff(self, func, inp):
        return grad(func, inp, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape, dtype=func.dtype).to(device))[0]

    def complex_mse(self, v1, v2):
        assert v1.shape == v2.shape
        assert v1.shape[1] == 1
        return F.mse_loss(v1.real, v2.real)+F.mse_loss(v2.imag, v2.imag)

    def add_imag_dim(self, v1):
        z = torch.zeros(v1.shape).requires_grad_(False).to(device)
        return torch.complex(v1, z)
    
cpinn = ComplexPhysicsInformedNN(model=complex_model, lb=lb, ub=ub, scale=False).to(device)
cpinn.load_state_dict(cpu_load("./saved_path_inverse_nls/NLS_cpinn.pth"))

<All keys matched successfully>

#### Goals
(1) Re-implement the semisup_model for a complex network.

(2) Implement the self.gradients function.
- complex_model(input) -> diff(u_pred, x) & diff(v_pred, x) -> combine 2 diff terms as 1 complex vector -> compute PDE loss / passing to the selector network

In [5]:
xx, tt = dimension_slicing(to_tensor(X_star, True))
predictions = complex_model(cat(xx, tt))
h = cplx2tensor(predictions)
h_x = complex_diff(predictions, xx)
h_xx = complex_diff(h_x, xx)
h_xxx = complex_diff(h_xx, xx)
h_t = complex_diff(predictions, tt)

In [6]:
f = 1j*h_t+0.5*h_xx+(h.abs()**2)*h

In [7]:
real_loss = (f.real**2).mean(); imag_loss = (f.imag**2).mean()
avg_loss = (real_loss+imag_loss)*0.5
avg_loss.item()

9.151408448815346e-06

In [8]:
derivatives = to_numpy(cat(h, h.abs()**2, h_x, h_xx, h_xxx))

dictionary = {}
for i in range(len(feature_names)): dictionary[feature_names[i]] = get_feature(derivatives, i)
dictionary

{'hf': array([[0.03347528+0.00087863j],
        [0.03361642+0.0007962j ],
        [0.03385198+0.00071853j],
        ...,
        [0.02741826+0.04537457j],
        [0.02729189+0.04540616j],
        [0.02720559+0.04547745j]], dtype=complex64),
 '|hf|': array([[0.00112137+0.j],
        [0.0011307 +0.j],
        [0.00114647+0.j],
        ...,
        [0.00281061+0.j],
        [0.00280657+0.j],
        [0.00280834+0.j]], dtype=complex64),
 'h_x': array([[ 0.00239915-0.00217538j],
        [ 0.00481033-0.00204157j],
        [ 0.00726255-0.00191126j],
        ...,
        [-0.00376286+0.00028983j],
        [-0.00272434+0.00133051j],
        [-0.00169292+0.00229347j]], dtype=complex64),
 'h_xx': array([[0.06119327+0.00345497j],
        [0.06225348+0.00338062j],
        [0.06330011+0.00329024j],
        ...,
        [0.02671357+0.02767968j],
        [0.02647803+0.02562424j],
        [0.02635859+0.02369945j]], dtype=complex64),
 'h_xxx': array([[ 0.02730165-0.00170198j],
        [ 0.02696816-0.00

In [9]:
c_poly = ComplexPolynomialFeatures(feature_names, dictionary)
complex_poly_features = c_poly.fit()
complex_poly_features

Computing hf
Computing |hf|
Computing h_x
Computing h_xx
Computing h_xxx
Computing hf^2
Computing hf |hf|
Computing hf h_x
Computing hf h_xx
Computing hf h_xxx
Computing |hf|^2
Computing |hf| h_x
Computing |hf| h_xx
Computing |hf| h_xxx
Computing h_x^2
Computing h_x h_xx
Computing h_x h_xxx
Computing h_xx^2
Computing h_xx h_xxx
Computing h_xxx^2


array([[ 1.0000000e+00+0.0000000e+00j,  3.3475280e-02+8.7863207e-04j,
         1.1213662e-03+0.0000000e+00j, ...,
         3.7326789e-03+4.2284193e-04j,  1.6765571e-03-9.8235323e-06j,
         7.4248313e-04-9.2933915e-05j],
       [ 1.0000000e+00+0.0000000e+00j,  3.3616424e-02+7.9619884e-04j,
         1.1306977e-03+0.0000000e+00j, ...,
         3.8640667e-03+4.2091092e-04j,  1.6860167e-03-4.0586194e-05j,
         7.2280248e-04-1.1415266e-04j],
       [ 1.0000000e+00+0.0000000e+00j,  3.3851981e-02+7.1853399e-04j,
         1.1464730e-03+0.0000000e+00j, ...,
         3.9960784e-03+4.1654496e-04j,  1.6937144e-03-7.1730574e-05j,
         7.0261088e-04-1.3404422e-04j],
       ...,
       [ 1.0000000e+00+0.0000000e+00j,  2.7418256e-02+4.5374572e-02j,
         2.8106126e-03+0.0000000e+00j, ...,
        -5.2549702e-05+1.4788458e-03j,  1.2934868e-03-1.6578888e-03j,
        -2.8707101e-03+8.2925614e-04j],
       [ 1.0000000e+00+0.0000000e+00j,  2.7291894e-02+4.5406163e-02j,
         2.8065671e-03

In [10]:
complex_poly_features.shape

(51456, 21)

In [11]:
w = TrainSTRidge(complex_poly_features, to_numpy(h_t), 1e-6, 1000)
print("PDE derived using STRidge")
print_pde(w, c_poly.poly_feature_names)

PDE derived using STRidge
u_t = (-0.000064 +0.499980i)h_xx
    + (-0.000131 +0.999982i)hf |hf|
   


In [12]:
c_poly.poly_feature_names

['1',
 'hf',
 '|hf|',
 'h_x',
 'h_xx',
 'h_xxx',
 'hf^2',
 'hf |hf|',
 'hf h_x',
 'hf h_xx',
 'hf h_xxx',
 '|hf|^2',
 '|hf| h_x',
 '|hf| h_xx',
 '|hf| h_xxx',
 'h_x^2',
 'h_x h_xx',
 'h_x h_xxx',
 'h_xx^2',
 'h_xx h_xxx',
 'h_xxx^2']